In [ ]:
#Task 5- cell 1
import cv2
import numpy as np

def degrade_frame(img_bgr, mode="original"):
    if mode == "original":
        return img_bgr

    h, w = img_bgr.shape[:2]

    if mode == "blur":
        # fairly strong blur
        return cv2.GaussianBlur(img_bgr, (9, 9), 2.5)

    elif mode == "down_up":
        # shrink then enlarge to simulate low-res
        small = cv2.resize(img_bgr, (w // 4, h // 4), interpolation=cv2.INTER_AREA)
        return cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    elif mode == "noise":
        # additive Gaussian noise
        noise = np.random.normal(0, 20, img_bgr.shape).astype(np.float32)
        noisy = img_bgr.astype(np.float32) + noise
        return np.clip(noisy, 0, 255).astype(np.uint8)

    else:
        return img_bgr


In [ ]:
#cell 2
from glob import glob
import os
import numpy as np

def compute_mae_for_mode(mode, conf_thresh=0.15, sigma=25):
    image_paths = sorted(glob("eval_frames/*.png") + glob("eval_frames/*.jpg"))
    pred_counts_mode = {}

    for img_path in image_paths:
        img_name = os.path.basename(img_path)
        img_bgr = cv2.imread(img_path)

        img_deg = degrade_frame(img_bgr, mode=mode)

        _, _, person_count = yolo_heatmap_on_image(
            img_deg,
            model,
            conf_thresh=conf_thresh,
            sigma=sigma
        )

        pred_counts_mode[img_name] = person_count

    # compute MAE vs ground truth
    errors = []
    for name, true_c in gt_counts.items():
        if name in pred_counts_mode:
            errors.append(abs(true_c - pred_counts_mode[name]))

    mae = float(np.mean(errors)) if errors else None
    return mae


In [ ]:
#cell 3
modes = ["original", "blur", "down_up", "noise"]
results = {}

for m in modes:
    mae_m = compute_mae_for_mode(m)
    results[m] = mae_m
    print(f"{m}: MAE = {mae_m:.2f}")

print("\nSummary:")
for m, v in results.items():
    print(f"{m:9s}  {v:.2f}")
